In [42]:
from docplex.mp.model import Model
import pandas as pd

# sava datas in a matrix of matrixes
first_data = [[0, 67, 23, 89], [56, 0, 45, 23], [12, 34, 0, 78], [78, 90, 12, 0]]

second_data = [[5, 5, 5, 5], [5, 5, 5, 5],[5, 5, 5, 5], [5, 5, 5, 5]]

third_data = [[0, 78, 12, 34], [23, 0, 67, 45], [1, 1, 1, 1], [1, 1, 1, 1]]

data = [first_data, second_data, third_data]

# parameters
c = 4  # number of nodes
d = [8, 9, 10, 11]  # time intervals
tw = [[8, 12], [8, 12], [8, 12]]  # time windows
v = 1  # number of vehicles
q = [0, 10, 10, -10]  # quantity of goods to collect/deliver
Q = 100  # maximum capacity of the vehicle
t_mv = 0.5  # time to move one item in/out from a vehicle [min]
M = 100000  # big M
tot_q = 0  # total quantity of goods to collect/delivery
del_q = 0  # quantity of goods to deliver

for good in q:
    tot_q += abs(good)

tot_t_mv = tot_q * t_mv

# print("Total time required to collect/deliver all goods:", tot_t_mv, "minutes")

for good in q:
    if good < 0:
        del_q += good
del_q = abs(del_q)

# define model
mdl = Model("CVRP")
mdl_energy = Model("CVRP_energy")

# decision variables
x = mdl.binary_var_cube(c, c, len(d) - 1, name="x")
T = mdl.continuous_var_list(c + 1, name="t")
C = mdl.continuous_var_list(c, name="C")

# energy model decision variables
x_energy = mdl_energy.binary_var_cube(c, c, v, name="x")
f = mdl_energy.continuous_var_cube(c, c, v, name="f")
w = mdl_energy.continuous_var_list(v, name="w")
C_energy = mdl_energy.continuous_var_list(c, name="C")

# nodes constraints
mdl.add_constraint(
    mdl.sum(mdl.sum(x[0, j, t] for j in range(1, c)) for t in range(len(d) - 1)) == 1
)

mdl.add_constraint(
    mdl.sum(mdl.sum(x[j, 0, t] for j in range(1, c)) for t in range(len(d) - 1)) == 1
)

for j in range(1, c):
    mdl.add_constraint(
        mdl.sum(
            mdl.sum(x[i, j, t] for i in range(c) if i != j) for t in range(len(d) - 1)
        )
        == 1
    )

for j in range(1, c):
    mdl.add_constraint(
        mdl.sum(
            mdl.sum(x[i, j, t] for i in range(c) if i != j) for t in range(len(d) - 1)
        )
        == mdl.sum(
            mdl.sum(x[j, i, t] for i in range(c) if i != j) for t in range(len(d) - 1)
        )
    )

# time constraints
for t in range(len(d) - 1):
    mdl.add_constraint(T[0] >= d[t] * 60 * mdl.sum(x[0, j, t] for j in range(1, c)))


for t in range(len(d) - 1):
    for j in range(1, c):
        mdl.add_constraint(T[j] >= d[t] * 60 * mdl.sum(x[i, j, t] for i in range(1, c)))

for t in range(len(d) - 1):
    for j in range(1, c):
        mdl.add_constraint(T[j]<= d[t+1] * 60 +(1- mdl.sum(x[i, j, t] for i in range(1, c)))*M)

for t in range(len(d) - 1):
    for i in range(1, c):
        mdl.add_constraint(T[i]>= d[t] * 60 * mdl.sum(x[i, j, t] for j in range(0, c)))

for t in range(len(d) - 1):
    for i in range(1, c):
        mdl.add_constraint(T[i]<= d[t+1] * 60 +(1- mdl.sum(x[i, j, t] for j in range(0, c)))*M)

for i in range(c):
    for j in range(1, c):
        for t in range(len(d) - 1):
            if i != j:
                mdl.add_constraint(
                    (1 - x[i, j, t]) * M + T[j] - T[i]
                    >= data[t][i][j] + t_mv * abs(q[j])
                )
                mdl.add_constraint(
                    (1 - x[i, j, t]) * M - T[j] + T[i]
                    >= -data[t][i][j] - t_mv * abs(q[j])
                )

for t in range(len(d) - 1):
    for i in range(1, c):
        mdl.add_constraint((1 - x[i, 0, t]) * M + T[c] - T[i] >= data[t][i][0])
        mdl.add_constraint((1 - x[i, 0, t]) * M - T[c] + T[i] >= -data[t][i][0])

for i in range(1, c):
    mdl.add_constraint(tw[i - 1][0] * 60 <= T[i])

for i in range(1, c):
    mdl.add_constraint(T[i] <= tw[i - 1][1] * 60)

mdl.add_constraint(T[c] - T[0] <= 8 * 60)

# quantity constraints
for t in range(len(d) - 1):
    for i in range(c):
        for j in range(1, c):
            if i != j:
                mdl.add_constraint(C[i] - C[j] + q[j] <= (1 - x[i, j, t]) * M)
                mdl.add_constraint(C[j] - C[i] - q[j] <= (1 - x[i, j, t]) * M)

for i in range(c):
    mdl.add_constraint(max(0, q[i]) <= C[i])
    mdl.add_constraint(C[i] <= min(Q, Q + q[i]))

mdl.add_constraint(C[0] >= del_q)

# objective function
mdl.minimize(
    mdl.sum(
        mdl.sum(mdl.sum(x[i, j, t] * data[t][i][j] for i in range(c)) for j in range(c))
        for t in range(len(d) - 1)
    )
)

# mdl.minimize(
#   T[c]-T[0]
# )

sol = mdl.solve()

sol_energy = mdl_energy.solve()

if sol_energy is None:
    print("No energy solution found")

if sol is None:
    print("No solution found")

print(sol.objective_value + tot_t_mv)

for t in range(len(d) - 1):
    for row in range(c):
            for col in range(c):
                if row==0 and sol.get_value(x[row, col, t]) == 1:
                    route=[(t,row, sol.get_value(T[row]))]
                    capacity=[(row,sol.get_value(C[row]))]
                    route.append((t,col, sol.get_value(T[col])))
                    capacity.append((col,sol.get_value(C[col])))
                elif col==0 and sol.get_value(x[row, col, t]) == 1:
                    end=((t,col, sol.get_value(T[c])))
                elif sol.get_value(x[row, col, t]) == 1:
                    route.append((t,col, sol.get_value(T[col])))
                    capacity.append((col,sol.get_value(C[col])))
route.append(end)

print(
    " -> ".join(
        f"{node} (arrivo: {arrival:.2f} , range temporale:{t})"
        for t,node, arrival in route
    )
)

print("\nCapacity for each node in order of visit:")
for node, cap in capacity:
    print(f"Node {node}: {cap:.2f}")


    
                    


Total time required to collect/deliver all goods: 15.0 minutes
27.0
0 (arrivo: 580.00 , range temporale:1) -> 1 (arrivo: 590.00 , range temporale:1) -> 3 (arrivo: 600.00 , range temporale:1) -> 2 (arrivo: 606.00 , range temporale:2) -> 0 (arrivo: 607.00 , range temporale:2)

Capacity for each node in order of visit:
Node 0: 10.00
Node 1: 20.00
Node 3: 10.00
Node 2: 20.00
